In [1]:
import sys, os, time
sys.path.append('/'.join(os.getcwd().split('/')))
import dataset
from util import init_params
from sampler import gaus
from dataset import eYaleB, MNIST
import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
# dd = eYaleB('/home/reddragon/data/face_detected_resized', label = False)
dd = MNIST('/home/reddragon/data/MNIST', train = False, label = False)

In [12]:
batch_size = 100
rand_sampler = torch.utils.data.RandomSampler(dd, replacement = True, num_samples = batch_size * 100)
train_data_loader = torch.utils.data.DataLoader(dd, batch_size, num_workers = 5, sampler = rand_sampler)

In [9]:
start = time.time()
for x in train_data_loader:
    pass
time.time() - start

0.2547485828399658

In [14]:
train_loader = torch.utils.data.DataLoader(dd, 2, num_workers = 5, shuffle = True, drop_last=True)

In [65]:
decay = 1.0
lamb = lambda e: decay * 1.0 * (0.5 ** (e >= 2)) * (0.2 ** (e >= 4)) * (0.1 ** (e >= 6))
net = nn.Sequential(
    nn.Linear(1,10),
    nn.Linear(10,1)
)
optimizer = optim.Adam(list(net.parameters()))
scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer, lamb)

In [66]:
for i in range(10):
    for param_group in optimizer.param_groups:
        print(param_group['lr'])
    scheduler.step()

0.001
0.001
0.0005
0.00025
2.5e-05
2.5e-06
2.5000000000000005e-08
2.500000000000001e-10
2.500000000000002e-12
2.5000000000000023e-14


In [15]:
x = next(iter(train_loader))

In [16]:
x.shape

torch.Size([2, 1, 28, 28])

In [18]:
d = 64
embed_data = nn.Sequential(
    nn.Conv2d(1, d, kernel_size = 5, stride = 2, padding = 2, bias = False),
    nn.BatchNorm2d(d),
    nn.ReLU(True),
)
embed_condition = nn.Sequential(
    nn.Linear(40, 64*64),
    nn.Unflatten(1, (1,64,64)),
)
enc = nn.Sequential(
    nn.Conv2d(d+1, 2*d, kernel_size = 5, stride = 2, padding = 2, bias = False),
    nn.BatchNorm2d(2*d),
    nn.ReLU(True),

    nn.Conv2d(2*d, 4*d, kernel_size = 5, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(4*d),
    nn.ReLU(True),

    nn.Conv2d(4*d, 8*d, kernel_size = 3, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(8*d),
    nn.ReLU(True),

    nn.Conv2d(8*d, 16*d, kernel_size = 3, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(16*d),
    nn.ReLU(True),

    nn.Flatten(),
    nn.Linear(16*16*d, 16)
    )

In [21]:
enc(torch.cat((embed_data(x), embed_condition(y)), dim = 1)).shape

torch.Size([2, 16])

In [38]:
d = 128
enc = nn.Sequential(
    nn.Conv2d(1, d, kernel_size = 4, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(d),
    nn.ReLU(True),

    nn.Conv2d(d, 2*d, kernel_size = 4, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(2*d),
    nn.ReLU(True),

    nn.Conv2d(2*d, 4*d, kernel_size = 4, stride = 2, padding = 2, bias = False),
    nn.BatchNorm2d(4*d),
    nn.ReLU(True),

    nn.Conv2d(4*d, 8*d, kernel_size = 4, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(8*d),
    nn.ReLU(True),

    nn.Flatten(),
    nn.Linear(4*8*d, 8),
    )

dec = nn.Sequential(
    nn.Linear(8, 49*8*d),
    nn.Unflatten(1, (8*d, 7, 7)),

    nn.ConvTranspose2d(8*d, 4*d, kernel_size = 4, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(4*d),
    nn.ReLU(True),

    nn.ConvTranspose2d(4*d, 2*d, kernel_size = 4, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(2*d),
    nn.ReLU(True),

    # reconstruction
    nn.Conv2d(2*d, 1, kernel_size = 4, padding = 'same'),
    nn.Tanh(),
    )
dec(enc(x)).shape

/home/reddragon/.conda/envs/torch/lib/python3.6/site-packages/torch/nn/modules/conv.py:443: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /opt/conda/conda-bld/pytorch_1628579264685/work/aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)


torch.Size([2, 1, 28, 28])

In [12]:
aa = next(iter(train_loader))

In [14]:
aa[1]

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.3889, -0.3889],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.1111,  0.1111],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,

In [5]:
dataset

<module 'dataset' from '/home/reddragon/projects/XAE/XAE/dataset.py'>

In [1]:
import _base_model

In [2]:
_base_model.test()

tensor([[-1.6208],
        [-1.3205]])


In [35]:
import torch
import torch.nn as nn
import itertools

In [14]:
aa = []
aa.append(nn.Conv2d(3, 3, kernel_size = 3))
aa.append(nn.Conv2d(3, 3, kernel_size = 3))

In [48]:
class test():
    def __init__(self):
        self.mse = nn.MSELoss()
        self.c1 = nn.Linear(1, 5)
        self.c2 = nn.Linear(5, 1)
        self.c3 = nn.Linear(5, 1)
        self.all_nets = [self.c1, self.c2]
        self.all_nets2 = [self.c3]

    def initialize(self):
        for net in self.all_nets:
            init_params(net)

    def cmse(self, x,y):
        return -self.mse(x,y)

    def all_params(self):
        # return list(self.c1.parameters()) + list(self.c2.parameters())
        return sum([list(net.parameters()) for net in self.all_nets], []) + sum([list(net.parameters()) for net in self.all_nets2], [])
        # return sum([list(net.parameters()) for net in self.all_nets],[])

cc = test()
cc.cmse(torch.zeros(3,1), torch.ones(3,1))


tensor(-1.)

In [51]:
class test2(nn.Module):
    def __init__(self):
        super(test2, self).__init__()
        self.c1 = nn.Linear(1, 5)
        self.c2 = nn.Linear(5, 1)
        self.c3 = nn.Linear(5, 1)

    def forward(self, x):
        return self.c2(self.c1(x))

In [ ]:
def penalty_loss(self, x, y, n):
        logPz = self.discriminate(x)
        logQz = self.discriminate(y)
        return self.bceloss(logPz, torch.zeros_like(logPz)) + self.bceloss(logQz, torch.ones_like(logQz))

In [3]:
disc = nn.Sequential(
        nn.Linear(8, 64),
        nn.ReLU(True),

        nn.Linear(64, 32),
        nn.ReLU(True),

        nn.Linear(32, 16),
        nn.ReLU(True),

        nn.Linear(16, 1),
        )

criterion = nn.BCEWithLogitsLoss()

In [36]:
nn.functional.sigmoid(torch.ones(5, 1) * -10.0) + 1e-15

tensor([[4.5398e-05],
        [4.5398e-05],
        [4.5398e-05],
        [4.5398e-05],
        [4.5398e-05]])

In [37]:
EPS = 1e-15
torch.log(nn.functional.sigmoid(torch.ones(5, 1) * -10.0) + 1e-15)

tensor([[-10.0000],
        [-10.0000],
        [-10.0000],
        [-10.0000],
        [-10.0000]])

In [39]:
criterion(torch.ones(5, 1) * -10.0, torch.ones(5,1))

tensor(10.0000)

In [41]:
criterion(torch.ones(10,1)*0.5, torch.ones(10,1))

tensor(0.4741)

In [79]:
-torch.log(torch.sigmoid(torch.ones(5, 1)) + 1e-15)

tensor([[0.3133],
        [0.3133],
        [0.3133],
        [0.3133],
        [0.3133]])

In [80]:
-torch.log(1 - torch.sigmoid(torch.ones(5, 1)) + 1e-15)

tensor([[1.3133],
        [1.3133],
        [1.3133],
        [1.3133],
        [1.3133]])

In [4]:
x = gaus(8, 8)
y = disc(x)
criterion(y, torch.zeros_like(y)) +

tensor(0.8216, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [26]:
y = torch.zeros_like(y) * -10.0

In [68]:
criterion

BCEWithLogitsLoss()

In [45]:
criterion(y, torch.ones_like(y)) * 2

tensor(1.3863)

In [81]:
criterion(torch.ones_like(y), torch.ones_like(y))

tensor(0.3133)

In [73]:
criterion(torch.ones_like(y), torch.zeros_like(y))

tensor(1.3133)

In [74]:
criterion(torch.ones_like(y), -torch.ones_like(y))

tensor(2.3133)

In [75]:
criterion(torch.ones_like(y), torch.ones_like(y))

tensor(0.3133)

In [76]:
criterion(-torch.ones_like(y), torch.ones_like(y))

tensor(1.3133)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [15]:
class MixedLayer(nn.Module):
    def __init__(self, p, q, i, prob = True):
        super(MixedLayer, self).__init__()
        self.p = p
        self.q = q
        self.i = i
        self.mu = nn.Linear(p, q)
        if prob:
            self.logvar = nn.Linear(p, q)
    def forward(self, x):
        if prob:
            torch.cat((self.lin(x[:, 0:self.p]), x[:, self.p:(self.p+self.i)]), axis = 1)
        return torch.cat((self.lin(x[:, 0:self.p]), x[:, self.p:(self.p+self.i)]), axis = 1)

In [16]:
bb = MixedLayer(4, 2, 2)
init_params(bb)

In [17]:
print([p for p in bb.parameters()])

[Parameter containing:
tensor([[ 0.0065,  0.0040, -0.0011,  0.0109],
        [ 0.0133,  0.0128,  0.0037, -0.0115]], requires_grad=True), Parameter containing:
tensor([0.1375, 0.1806], requires_grad=True)]


In [18]:
x = torch.rand(2,6)
x

tensor([[0.1543, 0.6921, 0.2273, 0.5626, 0.4695, 0.7117],
        [0.2035, 0.9041, 0.0019, 0.0801, 0.5793, 0.5400]])

In [19]:
bb(x)

tensor([[0.1471, 0.1858, 0.4695, 0.7117],
        [0.1433, 0.1939, 0.5793, 0.5400]], grad_fn=<CatBackward>)

In [52]:
tt = test2()
tt.state_dict()

OrderedDict([('c1.weight',
              tensor([[ 0.5197],
                      [ 0.2178],
                      [ 0.6304],
                      [-0.0735],
                      [-0.6911]])),
             ('c1.bias',
              tensor([ 0.0903, -0.9596,  0.9422,  0.2009, -0.7207])),
             ('c2.weight',
              tensor([[ 0.1804,  0.0569,  0.3982,  0.1495, -0.0191]])),
             ('c2.bias', tensor([0.1114])),
             ('c3.weight',
              tensor([[-0.0667, -0.3785,  0.1985,  0.2912,  0.1087]])),
             ('c3.bias', tensor([-0.1786]))])

In [39]:
cc.initialize()
cc.c1(torch.ones(2,1))

tensor([[0.1718, 0.1337, 0.1460, 0.1453, 0.0997],
        [0.1718, 0.1337, 0.1460, 0.1453, 0.0997]], grad_fn=<AddmmBackward>)

In [40]:
cc.all_nets[0](torch.ones(2,1))

tensor([[0.1718, 0.1337, 0.1460, 0.1453, 0.0997],
        [0.1718, 0.1337, 0.1460, 0.1453, 0.0997]], grad_fn=<AddmmBackward>)

In [32]:
list(aa[0].parameters())

[Parameter containing:
 tensor([[[[-0.1250, -0.0502,  0.1356],
           [ 0.0441, -0.0939,  0.1377],
           [-0.0796,  0.1658, -0.1213]],
 
          [[-0.1224,  0.1321, -0.0536],
           [-0.1582,  0.0461, -0.1887],
           [-0.0433, -0.1256,  0.0193]],
 
          [[ 0.0326,  0.1915, -0.1872],
           [-0.1299,  0.1529, -0.1083],
           [-0.1577,  0.1423, -0.0041]]],
 
 
         [[[-0.0793, -0.0233, -0.0613],
           [ 0.1125, -0.1068,  0.1370],
           [-0.1799, -0.1326,  0.0041]],
 
          [[-0.0294, -0.1305,  0.1242],
           [-0.0774,  0.1900, -0.0029],
           [ 0.0664,  0.1848,  0.0539]],
 
          [[ 0.1042, -0.1635,  0.1009],
           [ 0.0298, -0.0833,  0.1036],
           [ 0.0288,  0.0512, -0.1495]]],
 
 
         [[[-0.0080, -0.1605, -0.1426],
           [-0.1732,  0.0027, -0.1489],
           [ 0.1298, -0.0741,  0.1302]],
 
          [[-0.0244,  0.1439, -0.1660],
           [ 0.0415,  0.0733,  0.0670],
           [ 0.1632, -0.0364, 

In [29]:
list(aa[0].parameters()) + list(aa[1].parameters())

[Parameter containing:
 tensor([[[[-0.1250, -0.0502,  0.1356],
           [ 0.0441, -0.0939,  0.1377],
           [-0.0796,  0.1658, -0.1213]],
 
          [[-0.1224,  0.1321, -0.0536],
           [-0.1582,  0.0461, -0.1887],
           [-0.0433, -0.1256,  0.0193]],
 
          [[ 0.0326,  0.1915, -0.1872],
           [-0.1299,  0.1529, -0.1083],
           [-0.1577,  0.1423, -0.0041]]],
 
 
         [[[-0.0793, -0.0233, -0.0613],
           [ 0.1125, -0.1068,  0.1370],
           [-0.1799, -0.1326,  0.0041]],
 
          [[-0.0294, -0.1305,  0.1242],
           [-0.0774,  0.1900, -0.0029],
           [ 0.0664,  0.1848,  0.0539]],
 
          [[ 0.1042, -0.1635,  0.1009],
           [ 0.0298, -0.0833,  0.1036],
           [ 0.0288,  0.0512, -0.1495]]],
 
 
         [[[-0.0080, -0.1605, -0.1426],
           [-0.1732,  0.0027, -0.1489],
           [ 0.1298, -0.0741,  0.1302]],
 
          [[-0.0244,  0.1439, -0.1660],
           [ 0.0415,  0.0733,  0.0670],
           [ 0.1632, -0.0364, 

In [20]:
import math
math.log(0.5)*-1

0.6931471805599453

In [34]:
sum([list(net.parameters()) for net in aa], [])

[Parameter containing:
 tensor([[[[-0.1250, -0.0502,  0.1356],
           [ 0.0441, -0.0939,  0.1377],
           [-0.0796,  0.1658, -0.1213]],
 
          [[-0.1224,  0.1321, -0.0536],
           [-0.1582,  0.0461, -0.1887],
           [-0.0433, -0.1256,  0.0193]],
 
          [[ 0.0326,  0.1915, -0.1872],
           [-0.1299,  0.1529, -0.1083],
           [-0.1577,  0.1423, -0.0041]]],
 
 
         [[[-0.0793, -0.0233, -0.0613],
           [ 0.1125, -0.1068,  0.1370],
           [-0.1799, -0.1326,  0.0041]],
 
          [[-0.0294, -0.1305,  0.1242],
           [-0.0774,  0.1900, -0.0029],
           [ 0.0664,  0.1848,  0.0539]],
 
          [[ 0.1042, -0.1635,  0.1009],
           [ 0.0298, -0.0833,  0.1036],
           [ 0.0288,  0.0512, -0.1495]]],
 
 
         [[[-0.0080, -0.1605, -0.1426],
           [-0.1732,  0.0027, -0.1489],
           [ 0.1298, -0.0741,  0.1302]],
 
          [[-0.0244,  0.1439, -0.1660],
           [ 0.0415,  0.0733,  0.0670],
           [ 0.1632, -0.0364, 